# README

This notebook is designed to help you create a SIP/SIPREC integration with CCAI. You can execute this notebook in the "Colab Enterprise" product within the Google Cloud Console. Note that you will need to enable some additional apis/services to run Colab Enterprise.

This notebook is divided into 4 sections :
1. Project setup :
  - Specify your project details. If you are using a pre-generated JWT (from a Service Account for example) you can enter that here so that API calls are made using those credentials. We enable the Dialogflow API in the project using your credentials just in case it hasn't been done before in the UI.

2. Configuring CCAI for SIP/SIPREC :
  - Create a dummy Dialogflow ES agent and set the tier to 'Enterprise' in order to override the free 'Trial' quota that is set up by default.
  - Get a Google Phone Number, create a Security Settings object, create a ConversationProfile and associate the Google Phone Number with the ConversationProfile.

3. SIP Trunk Management :
  - Create a SIP TRUNK with your SBCs Fully Qualified Domain Names (FQDNs) as containted in the certificates used for authentication.
  - Other methods to manage the SIP TRUNKs.

4. Utility functions :
  - Get and List methods to view resources created.
  - Delete method to delete resources created.

# Project setup

In [1]:
import json
import os
import re
import sys


PROJECT_ID = !gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
REGION = "ccai-region" # @param {type:"string"}
JWT = "Leave as-is to use SSO or enter generated JWT token here" # @param {type:"string"}

# Ensure that the project has Dialogflow APIs enabled
!gcloud services enable dialogflow.googleapis.com

CONTAINS_SPACES_PATTERN = r"\s"
# If the JWT string has spaces, then use SSO for authentication
if re.search(CONTAINS_SPACES_PATTERN, JWT):
      JWT = !gcloud auth print-access-token
      JWT = JWT[0]

if REGION == "global":
    LOCATION_ID = ""
else:
    LOCATION_ID = REGION + "-"

# Configuring CCAI for SIP/SIPREC

In [ ]:
# @title Create a dummy Dialogflow ES agent and set the tier to Enterprise in order to remove the Trial quota limits and permit making additional quota requests

create_dummy_agent = f"""
curl -X POST \
-H "Authorization: Bearer {JWT}" \
-H "X-Goog-User-Project: {PROJECT_ID}" \
-H "Content-Type: application/json; charset=utf-8" \
-d '{{
        "displayName": "Dummy_ES_agent",
        "timeZone": "America/Los_Angeles",
        "tier": "TIER_ENTERPRISE"
    }}' \
https://dialogflow.googleapis.com/v2beta1/projects/{PROJECT_ID}/locations/global/agent
"""

request = os.popen(create_dummy_agent).read()

if isinstance(request, dict):
  response = json.loads(request)
  print(response)
else:
  print(request)

In [ ]:
# @title Script to create a ConversationProfile with GTP integration and export to Insights enabled

CONVERSATION_PROFILE_NAME = "GTP integration with export to Insights" # @param {type:"string"}
SECURITY_SETTINGS_NAME = "Export to Insights ONLY" # @param {type:"string"}
PHONE_NUMBER_COUNTRY_CODE = 1 # @param {type:"number"}
CX_AGENT_NAME = "CX agent name" # @param {type:"string"}


AGENT_NAME_PATTERN = r"^projects/([^/]+)/locations/([^/]+)/agents/([^/]+)"
if re.search(AGENT_NAME_PATTERN, CX_AGENT_NAME):
    print("Dialogflow CX agent name provided.")
    virtual_agent = True
else:
    print("No Dialogflow CX agent name provided.")
    virtual_agent = False

project_allowlisted = True

# Create PhoneNumberOrder

print("\nCreating Phone Number Order Object:\n")
create_phone_number_order = f"""
curl -X POST \
-H "Authorization: Bearer {JWT}" \
-H "X-Goog-User-Project: {PROJECT_ID}" \
-H "Content-Type: application/json; charset=utf-8" \
-d '{{
        "phoneNumberSpec": {{
            "countryCode": {PHONE_NUMBER_COUNTRY_CODE},
            "count": 1
        }}
    }}' \
https://{LOCATION_ID}dialogflow.googleapis.com/v2beta1/projects/{PROJECT_ID}/locations/{REGION}/phoneNumberOrders
"""

request = os.popen(create_phone_number_order).read()

if isinstance(request, str):
  phone_number_order_response = json.loads(request)
  phone_number_name = list(phone_number_order_response["phoneNumbers"].keys())[0]
  print("Phone number name: " + phone_number_name)
else:
  print(f'\n>>>> PROJECT "{PROJECT_ID}" HAS NOT BEEN ALLOWLISTED. <<<< \n\n\n'
        'Submit the form https://docs.google.com/forms/d/e/1FAIpQLSe2xMdpg_L2bhvl4EoNTUC4Nctzc3Qlw54uQ1_JNFgr0VhOfg/viewform '
        'wait to hear back and then re-run this script.\n\n')
  project_allowlisted = False

if project_allowlisted:

  # Create Security Settings object

  print("\nCreating Security Settings Object:\n")
  create_security_settings_obj = f"""
  curl -X POST \
  -H "Authorization: Bearer {JWT}" \
  -H "X-Goog-User-Project: {PROJECT_ID}" \
  -H "Content-Type: application/json; charset=utf-8" \
  -d '{{
          "displayName": "{SECURITY_SETTINGS_NAME}",
          "insightsExportSettings": {{
              "enableInsightsExport": true
          }}
      }}' \
  https://{LOCATION_ID}dialogflow.googleapis.com/v3/projects/{PROJECT_ID}/locations/{REGION}/securitySettings
  """

  security_settings_response = json.loads(os.popen(create_security_settings_obj).read())
  security_settings_name = security_settings_response["name"]
  print("Security settings name: " + security_settings_name)

  # Create ConversationProfile

  if virtual_agent:
    print("\nCreating Conversation Profile object with CX virtual agent and Insights support:\n")
    create_siprec_conversation_profile_obj = f"""
    curl -X POST \
    -H "Authorization: Bearer {JWT}" \
    -H "X-Goog-User-Project: {PROJECT_ID}" \
    -H "Content-Type: application/json; charset=utf-8" \
    -d '{{
            "displayName": "{CONVERSATION_PROFILE_NAME}",
            "languageCode": "en-US",
            "automatedAgentConfig": {{
                "agent": "{CX_AGENT_NAME}"
            }},
            "sipConfig": {{
                "createConversationOnTheFly": true,
                "allowVirtualAgentInteraction": true
            }},
            "securitySettings": "{security_settings_name}"
        }}' \
    https://{LOCATION_ID}dialogflow.googleapis.com/v2beta1/projects/{PROJECT_ID}/locations/{REGION}/conversationProfiles
    """

  else:
    print("\nCreating Conversation Profile object with transcription and Insights support.:\n")
    create_siprec_conversation_profile_obj = f"""
    curl -X POST \
    -H "Authorization: Bearer {JWT}" \
    -H "X-Goog-User-Project: {PROJECT_ID}" \
    -H "Content-Type: application/json; charset=utf-8" \
    -d '{{
            "displayName": "{CONVERSATION_PROFILE_NAME}",
            "languageCode": "en-US",
            "sipConfig": {{
                "createConversationOnTheFly": true
            }},
            "securitySettings": "{security_settings_name}"
        }}' \
    https://{LOCATION_ID}dialogflow.googleapis.com/v2beta1/projects/{PROJECT_ID}/locations/{REGION}/conversationProfiles
    """

  siprec_conversation_profile_response = json.loads(os.popen(create_siprec_conversation_profile_obj).read())
  conversation_profile_obj_name = siprec_conversation_profile_response["name"]
  print("Conversation profile name: " + conversation_profile_obj_name)

  # Associate PhoneNumber with ConversationProfile

  print("\nAssociating Phone Number with Conversation Profile:\n")
  associate_phone_number_with_conv_profile = f"""
  curl -X PATCH \
  -H "Authorization: Bearer {JWT}" \
  -H "X-Goog-User-Project: {PROJECT_ID}" \
  -H "Content-Type: application/json; charset=utf-8" \
  -d '{{
          "conversationProfile": "{conversation_profile_obj_name}"
      }}' \
  https://{LOCATION_ID}dialogflow.googleapis.com/v2beta1/{phone_number_name}?update_mask=conversationProfile
  """

  response = json.loads(os.popen(associate_phone_number_with_conv_profile).read())
  print(response)

print("\nDone!")

# SIP Trunk Creation

In [ ]:
# @title  CreateSipTrunk

HOSTNAMES = ["host.na.me1", "host.na.me2"] # @param
DISPLAY_NAME = "sip_trunk_name" # @param {type:"string"}


hostname_expression = ""
for host in HOSTNAMES:
    hostname_expression += f'"expected_hostname": "{host}",'

curl = f"""
curl -X POST \
-H "Authorization: Bearer {JWT}" \
-H "X-Goog-User-Project: {PROJECT_ID}" \
-H "Content-Type: application/json; charset=utf-8" \
-d '{{
        {hostname_expression}
        "display_name": "{DISPLAY_NAME}"
    }}' \
https://{LOCATION_ID}dialogflow.googleapis.com/v2beta1/projects/{PROJECT_ID}/locations/{REGION}/sipTrunks
"""

request = os.popen(curl).read()
if isinstance(request, dict):
  response = json.loads(request)
  print(response)
else:
  print(request)

# Utility functions (List/Get/Delete methods for resources created)

In [ ]:
# @title View ConversationProfile and PhoneNumber details

conversation_profiles_dict = {}
orphan_phone_numbers = []

# Get all the ConversationProfiles and see which ones have sipConfig configured
curl = f"""
curl -X GET \
-H "Authorization: Bearer {JWT}" \
-H "X-Goog-User-Project: {PROJECT_ID}" \
-H "Content-Type: application/json; charset=utf-8" \
https://{LOCATION_ID}dialogflow.googleapis.com/v2beta1/projects/{PROJECT_ID}/locations/{REGION}/conversationProfiles
"""

request = os.popen(curl).read()
if isinstance(request, str):
  response = json.loads(request)

  for data in response["conversationProfiles"]:
    conversation_profile_name = data["name"]
    conversation_profiles_dict[conversation_profile_name] = {}
    # print(data)
    if "sipConfig" in data:
      conversation_profiles_dict[conversation_profile_name]["sipConfig"] = data["sipConfig"]
    if "automatedAgentConfig" in data:
      conversation_profiles_dict[conversation_profile_name]["automatedAgentConfig"] = data["automatedAgentConfig"]

# Get all phone numbers and see which ones are associated with a ConversationProfile
curl = f"""
curl -X GET \
-H "Authorization: Bearer {JWT}" \
-H "X-Goog-User-Project: {PROJECT_ID}" \
-H "Content-Type: application/json; charset=utf-8" \
https://{LOCATION_ID}dialogflow.googleapis.com/v2beta1/projects/{PROJECT_ID}/locations/{REGION}/phoneNumbers
"""

request = os.popen(curl).read()
if isinstance(request, str):
  response = json.loads(request)

  if "phoneNumbers" in response:
    for data in response["phoneNumbers"]:
      phone_number = data["phoneNumber"]
      if "conversationProfile" in data:
        conversation_profile_name = data["conversationProfile"]
        conversation_profiles_dict[conversation_profile_name]["phone_number"] = phone_number
      else:
        orphan_phone_numbers.append(phone_number)

    if orphan_phone_numbers:
      print(f"There is/are {len(orphan_phone_numbers)} phone numbers not associated with a ConversationProfile:\n {orphan_phone_numbers}\n\n")

    for conversation_profile_name, details in conversation_profiles_dict.items():
      print(f"Conversation Profile: {conversation_profile_name}")
      if details:
        if "sipConfig" in details and details["sipConfig"]:
          print(f"SipConfig is set: {details['sipConfig']}")
        if "automatedAgentConfig" in details:
          print(f"Virtual agent is configured: {details['automatedAgentConfig']}")
        if "phone_number" in details:
          print(f"Phone number is associated: {details['phone_number']}")
      print("\n")
  else:
    print("No phone numbers present")

In [ ]:
# @title GetConversation

conversation_name = "projects/<project-id>/locations/<location>/conversations/<conversation-name>" # @param {type:"string"}

curl = f"""
curl -X GET \
-H "Authorization: Bearer {JWT}" \
-H "X-Goog-User-Project: {PROJECT_ID}" \
-H "Content-Type: application/json; charset=utf-8" \
https://{LOCATION_ID}dialogflow.googleapis.com/v2beta1/{conversation_name}
"""

request = os.popen(curl).read()
if isinstance(request, dict):
  response = json.loads(request)
  print(response)
else:
  print(request)

In [ ]:
# @title ListSecuritySettings

curl = f"""
curl -X GET \
-H "Authorization: Bearer {JWT}" \
-H "X-Goog-User-Project: {PROJECT_ID}" \
-H "Content-Type: application/json; charset=utf-8" \
https://{LOCATION_ID}dialogflow.googleapis.com/v3/projects/{PROJECT_ID}/locations/{REGION}/securitySettings
"""

request = os.popen(curl).read()
if isinstance(request, dict):
  response = json.loads(request)
  print(response)
else:
  print(request)

In [ ]:
# @title  ListConversationProfiles

curl = f"""
curl -X GET \
-H "Authorization: Bearer {JWT}" \
-H "X-Goog-User-Project: {PROJECT_ID}" \
-H "Content-Type: application/json; charset=utf-8" \
https://{LOCATION_ID}dialogflow.googleapis.com/v2beta1/projects/{PROJECT_ID}/locations/{REGION}/conversationProfiles
"""

request = os.popen(curl).read()
if isinstance(request, dict):
  response = json.loads(request)
  print(response)
else:
  print(request)

In [ ]:
# @title ListPhoneNumbers

curl = f"""
curl -X GET \
-H "Authorization: Bearer {JWT}" \
-H "X-Goog-User-Project: {PROJECT_ID}" \
-H "Content-Type: application/json; charset=utf-8" \
https://{LOCATION_ID}dialogflow.googleapis.com/v2beta1/projects/{PROJECT_ID}/locations/{REGION}/phoneNumbers
"""

request = os.popen(curl).read()
if isinstance(request, dict):
  response = json.loads(request)
  print(response)
else:
  print(request)

In [15]:
# @title Delete resource objects

resource_name = "projects/<project-id>/locations/<location>/<resource>/<resource-id>" # @param {type:"string"}

security_settings_name = ""
conversation_profile_name = ""
phone_number_name = ""


security_settings_pattern = "^projects/([^/]+)/locations/([^/]+)/securitySettings/([^/]+)"
# Security settings uses the v3alpha1 api
if re.search(security_settings_pattern, resource_name):
    print("Deleting Security Settings object: " + resource_name)
    curl = f"""
    curl -X DELETE \
    -H "Authorization: Bearer {JWT}" \
    -H "X-Goog-User-Project: {PROJECT_ID}" \
    -H "Content-Type: application/json; charset=utf-8" \
    https://{LOCATION_ID}dialogflow.googleapis.com/v3alpha1/{resource_name}
    """
    response = json.loads(os.popen(curl).read())
    print(response)

conversation_profile_pattern = "^projects/([^/]+)/locations/([^/]+)/conversationProfiles/([^/]+)"
phone_number_pattern = "^projects/([^/]+)/locations/([^/]+)/phoneNumbers/([^/]+)"

# ConversationProfile and PhoneNumbers use the v2beta1 api
if (re.search(conversation_profile_pattern, resource_name) or
    re.search(phone_number_pattern, resource_name)):
    print("Deleting object: " + resource_name)
    curl = f"""
    curl -X DELETE \
    -H "Authorization: Bearer {JWT}" \
    -H "X-Goog-User-Project: {PROJECT_ID}" \
    -H "Content-Type: application/json; charset=utf-8" \
    https://{LOCATION_ID}dialogflow.googleapis.com/v2beta1/{resource_name}
    """
    response = json.loads(os.popen(curl).read())
    print(response)

In [ ]:
# @title  ListSipTrunks

curl = f"""
curl -X GET \
-H "Authorization: Bearer {JWT}" \
-H "X-Goog-User-Project: {PROJECT_ID}" \
-H "Content-Type: application/json; charset=utf-8" \
https://{LOCATION_ID}dialogflow.googleapis.com/v2beta1/projects/{PROJECT_ID}/locations/{REGION}/sipTrunks
"""

request = os.popen(curl).read()
if isinstance(request, dict):
  response = json.loads(request)
  print(response)
else:
  print(request)

In [17]:
# @title  GetSipTrunk

RESOURCE_NAME = "projects/<project-id>/locations/<location>/sipTrunks/<sip-trunk-id>" # @param {type:"string"}
# eg. value projects/project_name/locations/global/sipTrunks/JxN1ndLCR9OJvgrDlc32_Q

curl = f"""
curl -X GET \
-H "Authorization: Bearer {JWT}" \
-H "X-Goog-User-Project: {PROJECT_ID}" \
-H "Content-Type: application/json; charset=utf-8" \
https://{LOCATION_ID}dialogflow.googleapis.com/v2beta1/{RESOURCE_NAME}
"""

request = os.popen(curl).read()
if isinstance(request, dict):
  response = json.loads(request)
  print(response)
else:
  print(request)

In [ ]:
# @title  DeleteSipTrunk

RESOURCE_NAME = "projects/<project-id>/locations/<location>/sipTrunks/<sip-trunk-id>" # @param {type:"string"}
# eg. value projects/project_name/locations/global/sipTrunks/JxN1ndLCR9OJvgrDlc32_Q


curl = f"""
curl -X DELETE \
-H "Authorization: Bearer {JWT}" \
-H "X-Goog-User-Project: {PROJECT_ID}" \
-H "Content-Type: application/json; charset=utf-8" \
https://{LOCATION_ID}dialogflow.googleapis.com/v2beta1/{RESOURCE_NAME}
"""

request = os.popen(curl).read()
if isinstance(request, dict):
  response = json.loads(request)
  print(response)
else:
  print(request)